<center> 
    <h1> Transformer TTS: A Text-to-Speech Transformer in TensorFlow 2 </h1>
    <h2> Audio synthesis with Forward Transformer TTS and MelGAN Vocoder</h2>
</center>

## Forward Model

In [1]:
# Clone the Transformer TTS and MelGAN repos
!git clone https://github.com/as-ideas/TransformerTTS.git
!git clone https://github.com/seungwonpark/melgan.git

Cloning into 'TransformerTTS'...
remote: Enumerating objects: 4114, done.
remote: Counting objects: 100% (653/653), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 4114 (delta 460), reused 612 (delta 433), pack-reused 3461
Receiving objects: 100% (4114/4114), 26.01 MiB | 19.02 MiB/s, done.
Resolving deltas: 100% (2830/2830), done.
Cloning into 'melgan'...
remote: Enumerating objects: 396, done.
remote: Total 396 (delta 0), reused 0 (delta 0), pack-reused 396
Receiving objects: 100% (396/396), 18.04 MiB | 25.90 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [2]:
# Install requirements
!apt-get install -y espeak
!pip install -r TransformerTTS/requirements.txt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 40 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak amd64 1.48.04+dfsg-5 [61.6 kB]
Fetched 1,219 kB in 1s (1,147 kB/s)
S

In [3]:
!cd TransformerTTS/; git checkout c3405c53e435a06c809533aa4453923469081147

Note: checking out 'c3405c53e435a06c809533aa4453923469081147'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at c3405c5 Fix path.


In [4]:
# Set up the paths
from pathlib import Path
MelGAN_path = 'melgan/'
TTS_path = 'TransformerTTS/'

import sys
sys.path.append(TTS_path)

In [5]:
# Load pretrained model
from model.factory import tts_ljspeech
from data.audio import Audio

model, config = tts_ljspeech()
audio = Audio(config)

175644672/175635288 [==============================] - 8s 0us/step


In [6]:
# Synthesize text
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out_normal = model.predict(sentence)

In [7]:
# Convert spectrogram to wav (with griffin lim)
wav = audio.reconstruct_waveform(out_normal['mel'].numpy().T)

In [8]:
import IPython.display as ipd

ipd.display(ipd.Audio(wav, rate=config['sampling_rate']))

You can also vary the speech speed

In [9]:
# 20% faster
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence, speed_regulator=1.20)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config['sampling_rate']))

In [10]:
# 10% slower
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence, speed_regulator=.9)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config['sampling_rate']))

### MelGAN

In [11]:
# Do some sys cleaning
sys.path.remove(TTS_path)
sys.modules.pop('model')

<module 'model' from 'TransformerTTS/model/__init__.py'>

In [12]:
sys.path.append(MelGAN_path)
import torch
import numpy as np

vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')
vocoder.eval()

mel = torch.tensor(out_normal['mel'].numpy().T[np.newaxis,:,:])

Downloading: "https://github.com/seungwonpark/melgan/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/seungwonpark/melgan/releases/download/v0.3-alpha/nvidia_tacotron2_LJ11_epoch6400.pt" to /root/.cache/torch/hub/checkpoints/nvidia_tacotron2_LJ11_epoch6400.pt


In [13]:
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)

In [14]:
# Display audio
ipd.display(ipd.Audio(audio.cpu().numpy(), rate=22050))